### Исполнитель: Рогов Илья ###
*Дата: 07/02/2024* 


Описание: Ниже представлены решения для каждого под задания. Для корректного запуска рекомендуется использовать следующие библиотеки:
* pandas
* request
* base64
* faker
* random
* ipython
* ipython_sql

Так же в заданиях 3, 4, 5 были сгенерированы случайные данные и загружены в базу данных ___test_database.db___.

### Задание 1 ###
 Для первого задания тебе необходимо скачать файл (в приложении)
Далее нужно прочитать таблицы с обоих листов используя библиотеку pandas. Из таблицы на втором листе, нужно удалить дубликаты для колонки Player, оставив в колонке Date наибольшую дату. После объединить обе таблицы, добавить колонку User с твоим почтым адресом.
Таким образом, полученная таблица будет иметь колонки Player, Score, Date, User.


In [1]:
import pandas as pd 

df_1 = pd.read_excel('test_data.xlsx', sheet_name=0) # Получаем данные из 1 листа из файла 'test_data.xlsx' и сохраняем в df_1
df_2 = pd.read_excel('test_data.xlsx', sheet_name=1) # Получаем данные из 2 листа из файла 'test_data.xlsx' и сохраняем в df_2


In [2]:
# Сортируем df_2 по дате и удаляем дубликаты в 'Player'
df_2 = df_2.sort_values('Date').drop_duplicates('Player', keep='last') 

# Объединяем df_1 и df_2 по 'Player' и сохраняем результат в df_result
df_result = pd.merge(df_1, df_2, on='Player')

# Добавляем в df_result новый столбец 'User' со значением почты
df_result['User'] = 'ilya.rogov@valuemont.com'
print(df_result)

        Player  Score       Date                      User
0         Dino    473 2022-02-10  ilya.rogov@valuemont.com
1         Bear    401 2022-01-30  ilya.rogov@valuemont.com
2     Elephant    337 2022-02-09  ilya.rogov@valuemont.com
3  Marshmallow    145 2022-02-07  ilya.rogov@valuemont.com
4      Unicorn    222 2022-02-09  ilya.rogov@valuemont.com
5        Doggo    500 2022-01-27  ilya.rogov@valuemont.com
6        Panda     58 2022-02-09  ilya.rogov@valuemont.com


### Задание 2 ###
Следующий шаг - авторизация в Anaplan. Здесь тебе поможет твои credentials, библиотека requests и документация от Anaplan Результат: AnaplanAuthToken, который будет использоваться для дальнейших запросов.


In [3]:
import requests
import base64

In [4]:
# Задаем учетные данные 
username = 'ilya.rogov@valuemont.com'
password = 'Gratop906kik'

# Задаем URL 
url = 'https://auth.anaplan.com/token/authenticate'

# Задаем header для запроса. Учитываем что Authorization закодирован в base64
headers = {
    'Authorization': 'Basic ' + base64.b64encode((username + ':' + password).encode('utf-8')).decode('utf-8'),
    'Content-Type': 'application/json'
}

In [5]:
# Отправляем POST запрос
response = requests.post(url, headers=headers)

# Если авторизация успешна выводим токен (tokenValue), иначе выводим статус запроса и текст
if 200 <= response.status_code < 300:
    anaplan_auth_token = response.json()['tokenInfo']['tokenValue']
    print('AnaplanAuthToken:', anaplan_auth_token)
else:
    print('Error:', response.status_code, response.text)


AnaplanAuthToken: Cx2WITUJT0TynZv3Erb2Tg==.dbbXZ0hfJAqougeSWCqkdZ48fvnG8L/vgNl92019+wlLZZi2CH781wPvWT+TOsVPdw/bpStaeUdCPnp/gVeusSkAc6SPsnvqzM3DXrM+KjmAmSixeg3Me7zPR6sO6/wIPB3JFzcDfDZ+68ve0DAwcr2PIP4dyPWbiFmtbiiI5tHnNsMt5mvXN6RYKQkWTr3KupFEYur94Z4W9uOEUkmafX9whnVox6DAa4bGjuPZKtnmWEePnjgCZxvEwgEpbgL+pDOpOyKkHeDUJrd3BSJnPJjcA2OXNTfA+ppRZgHZQrDQd3PBI7b/H5Mhkjl+nEZtgNGIyDTS3GLjeVQ9ebT2sFVqOQBe8qBtnA3rC24UHNRFtD/40y8Nc/o3SUOXg8gcS/BS83EWtQLRVEUYe8UeDGHITUCrTmIfhpMHpsYV2WhjHwrtVzdpKO6LymeEFeUXQmaRKQNTevBbQ5acTCqlzck/ptNqHYCeGaBRTXu4odDCtiT3dKeF/jNASPbiFR6YMeZuPGCYwxovSb1KVKUDLXGST3qyLIxVp8wOb2+SfZQ=.hO0jjT9RdXVkq7JHR89/Su24biz6Pe+N5GWF2rkxJS8=


### Задание 3 ### 
У вас есть база данных с таблицей "employees", содержащей следующие столбцы:
employee_id (целочисленный идентификатор сотрудника)
first_name (имя сотрудника)
last_name (фамилия сотрудника)
salary (заработная плата сотрудника)
Ваша задача:
Напишите SQL-запрос для выбора всех сотрудников, у которых заработная плата выше 50000.
Отсортируйте результат по убыванию заработной платы.
Выведите первые 5 строк результата.


In [6]:
# Следующая ячейка используются для создания базы данных (для наглядной визуализиции SQL запросов)
# Импорт библиотеки faker (позволяет заполнить БД случайной информацией) 
from faker import Faker

# Определение функции создания данных работников
def create_employees_info():
    fake = Faker() # Создание объекта Faker
    employees = [] # Создание пустого списка для хранения информации о сотрудниках
    for employee in range(50): # Цикл для генерации информации о 50 сотрудниках
        employees.append({ 
            "employee_id": fake.unique.random_number(digits=5), # уникальный ID
            "first_name": fake.first_name(), # Генерация имени
            "last_name": fake.last_name(), # Генерация Фамилии
            "salary": fake.random_int(min=30000, max=100000, step=1000) # Генерация зарплаты
        })
    return pd.DataFrame(employees) # Преобразуем список в DataFrame и возвращаем его

df_employees = create_employees_info() # Присваиваем df_employees результат функции create_employees_info() 

import sqlite3

conn =  sqlite3.connect('test_database.db') # Создаем/подключаем БД

df_employees.to_sql('employees', conn, if_exists='replace', index=False) # Конвертируем df_employees в БД SQL


In [8]:
# Загружаем расширение SQL для IPython
%load_ext sql

# Устанавливаем соединение с БД
%sql sqlite:///test_database.db


#### Решение ####

In [9]:
%%sql 
SELECT * 
FROM employees
WHERE salary > 50000
ORDER BY salary DESC
LIMIT 5

 * sqlite:///test_database.db
Done.


employee_id,first_name,last_name,salary
40969,Eugene,Maxwell,99000
97511,Bradley,Aguirre,97000
13383,Ivan,Underwood,97000
55808,Bradley,Patel,93000
3008,Lindsey,Porter,93000


### Задание 4 ###
Напишите SQL-запрос для нахождения второго максимального значения в столбце в таблице.


#### Решение ####

In [10]:
%%sql
SELECT MAX(salary) 
FROM employees
WHERE salary < (SELECT MAX(salary) FROM employees)

 * sqlite:///test_database.db
Done.


MAX(salary)
97000


### Задание 5 ###
Представим, у вас есть таблица "transactions" с полями "user_id", "amount", и "timestamp". Напишите SQL-запрос для поиска пользователей, у которых сумма транзакций за последний месяц превысила 1000.


In [25]:
# Следующая ячейка используются для создания базы данных (для наглядной визуализиции SQL запросов)
from random import randint

# Определение функции создания данных транзакций с заранее заданым количеством пользователей и количеством транзакций
def create_transactions_data(num_users=15, num_transactions=50):
    fake = Faker()
    transactions = []
    for transaction in range(num_transactions):
        user_id = randint(1, num_users) # Генерирация случайного ID
        amount = randint(1, 1000)  # Генерация случайной сумму транзакции
        timestamp = fake.date_between(start_date='-40d', end_date='today')  # Генерация случайной даты транзакции
        transactions.append({
            "user_id": user_id,
            "amount": amount,
            "timestamp": timestamp
        })
    return pd.DataFrame(transactions)

df_transactions = create_transactions_data() # Преобразуем список в DataFrame и возвращаем его


df_transactions.to_sql('transactions', conn, if_exists='replace', index=False) # Конвертируем df_transactions в БД SQL

50

#### Решение ####

In [26]:
%%sql 
SELECT user_id, SUM(amount) as total_amount
FROM transactions
WHERE timestamp > date('now','-1 month')
GROUP BY user_id
HAVING total_amount > 1000

 * sqlite:///test_database.db
Done.


user_id,total_amount
1,3493
2,1536
4,1294
5,1394
6,3391
9,1845
11,1010
12,1881
